In [1]:
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
import tensorflow as tf
from keras.metrics import Precision, Recall
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional, Dropout, GlobalMaxPooling1D, \
Concatenate, BatchNormalization, Conv1D, ReLU, Input, GRU
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.metrics import confusion_matrix
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras import regularizers
from keras.layers import Embedding, Flatten, Dense
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk import word_tokenize, pos_tag, ne_chunk
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from datetime import datetime

2024-02-01 01:42:25.252914: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/rusab1/Work/Personal_Projects/learntorch/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
games = pd.read_csv('datasets/steam_recommendation/games.csv')
users = pd.read_csv('datasets/steam_recommendation/users.csv').head()
recommendations = pd.read_csv('datasets/steam_recommendation/recommendations.csv')

games, users

(        app_id                                      title date_release   win  \
 0        13500          Prince of Persia: Warrior Within™   2008-11-21  True   
 1        22364                    BRINK: Agents of Change   2011-08-03  True   
 2       113020               Monaco: What's Yours Is Mine   2013-04-24  True   
 3       226560                         Escape Dead Island   2014-11-18  True   
 4       249050                    Dungeon of the ENDLESS™   2014-10-27  True   
 ...        ...                                        ...          ...   ...   
 50867  2296380  I Expect You To Die 3: Cog in the Machine   2023-09-28  True   
 50868  1272080                                   PAYDAY 3   2023-09-21  True   
 50869  1402110                                 Eternights   2023-09-11  True   
 50870  2272250                        Forgive Me Father 2   2023-10-19  True   
 50871  2488510                                  FatalZone   2023-10-23  True   
 
          mac  linux      

In [4]:
main_df = recommendations.merge(games, on='app_id')

In [17]:
train_dataset = main_df[['is_recommended', 'helpful', 'funny', 'hours','positive_ratio','rating','linux','mac','win','price_final','price_original','discount','app_id']]
train_dataset.columns

Index(['is_recommended', 'helpful', 'funny', 'hours', 'positive_ratio',
       'rating', 'linux', 'mac', 'win', 'price_final', 'price_original',
       'discount', 'app_id'],
      dtype='object')

In [ ]:
main_df.drop(columns=['review_id','user_id','date','app_id','user'], inplace=True)

In [18]:
arr= ['is_recommended', 'linux','mac','win']
train_dataset['rating'] = LabelEncoder().fit_transform(train_dataset['rating'])
for columns in arr:
    train_dataset[columns] = to_categorical(train_dataset[columns])

train_dataset.dropna(inplace=True)
train_dataset

/var/folders/mc/529dnmr107lbrjvk8tdft4y00000gq/T/ipykernel_67620/1719737818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset['rating'] = LabelEncoder().fit_transform(train_dataset['rating'])
/var/folders/mc/529dnmr107lbrjvk8tdft4y00000gq/T/ipykernel_67620/1719737818.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_dataset[columns] = to_categorical(train_dataset[columns])
/var/folders/mc/529dnmr107lbrjvk8tdft4y00000gq/T/ipykernel_67620/1719737818.py:6: SettingWithCopyWarning: 
A val

,is_recommended,helpful,funny,hours,positive_ratio,rating,linux,mac,win,price_final,price_original,discount,app_id
0,0.0,0,0,36.3,95,5,0.0,1.0,1.0,30.00,0.00,0.0,975370
1,0.0,0,0,24.3,95,5,0.0,1.0,1.0,30.00,0.00,0.0,975370
2,0.0,3,0,177.0,95,5,0.0,1.0,1.0,30.00,0.00,0.0,975370
3,0.0,5,0,388.6,95,5,0.0,1.0,1.0,30.00,0.00,0.0,975370
4,0.0,0,0,70.1,95,5,0.0,1.0,1.0,30.00,0.00,0.0,975370
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41154789,0.0,4,0,13.0,87,8,0.0,1.0,1.0,7.49,14.99,50.0,1643850
41154790,0.0,0,0,18.0,87,8,0.0,1.0,1.0,7.49,14.99,50.0,1643850
41154791,0.0,0,0,6.0,87,8,0.0,1.0,1.0,7.49,14.99,50.0,1643850
41154792,0.0,0,0,5.0,87,8,0.0,1.0,1.0,7.49,14.99,50.0,1643850


In [44]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(13,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

TypeError: Invalid keyword argument(s) in `compile()`: ({'validation_split'},). Valid keyword arguments include "cloning", "experimental_run_tf_function", "distribute", "target_tensors", or "sample_weight_mode".

In [26]:
# train_dataset = train_dataset.astype('float32')
# train_dataset.dtypes
np.savez_compressed('datasets/preprocessed_data/steam_game_recommendation/train_dataset.npz', data_1=train_dataset,columns=train_dataset.columns)

In [28]:
load = np.load("datasets/preprocessed_data/steam_game_recommendation/train_dataset.npz",allow_pickle=True)
load.files
data = load['data_1']
columns = load['columns']
#data
df = pd.DataFrame(data,columns=columns)
df
#train_dataset

,is_recommended,helpful,funny,hours,positive_ratio,rating,linux,mac,win,price_final,price_original,discount,app_id
0,0.0,0.0,0.0,36.3,95.0,5.0,0.0,1.0,1.0,30.00,0.00,0.0,975370.0
1,0.0,0.0,0.0,24.3,95.0,5.0,0.0,1.0,1.0,30.00,0.00,0.0,975370.0
2,0.0,3.0,0.0,177.0,95.0,5.0,0.0,1.0,1.0,30.00,0.00,0.0,975370.0
3,0.0,5.0,0.0,388.6,95.0,5.0,0.0,1.0,1.0,30.00,0.00,0.0,975370.0
4,0.0,0.0,0.0,70.1,95.0,5.0,0.0,1.0,1.0,30.00,0.00,0.0,975370.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41154789,0.0,4.0,0.0,13.0,87.0,8.0,0.0,1.0,1.0,7.49,14.99,50.0,1643850.0
41154790,0.0,0.0,0.0,18.0,87.0,8.0,0.0,1.0,1.0,7.49,14.99,50.0,1643850.0
41154791,0.0,0.0,0.0,6.0,87.0,8.0,0.0,1.0,1.0,7.49,14.99,50.0,1643850.0
41154792,0.0,0.0,0.0,5.0,87.0,8.0,0.0,1.0,1.0,7.49,14.99,50.0,1643850.0


In [42]:
real_label = train_dataset['is_recommended']
real_label

2

In [43]:
model.fit(train_npz, real_label, epochs=10, validation_split=0.3)

Epoch 1/10
   5934/1286088 [..............................] - ETA: 48:39 - loss: 69.3504 - accuracy: 0.1418

KeyboardInterrupt: 